In [489]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.neighbors import NearestNeighbors
from scipy.optimize import bisect, fixed_point, newton
from scipy.stats import entropy
from sklearn.datasets import make_swiss_roll
from sklearn.decomposition import PCA
from scipy.misc import logsumexp
from itertools import product

In [75]:
# choose NN, k (k<NN), ks  (ks < k)
# steps :
# - construct distance matrix between pairs of points for Y and Z
# - for a given a, b with a > 0, b > 0:
# - construct p based on the mixture-distance as_j^Y + bs_j^Z for each row i
# - find NN first neighbours for Y and Z then use it along with p to 
#   compute gaa, gab, gbb for each distribution i (Corrolary 10)
# - compute tau_i for Z and for Y (binary search for each distrib i to have entropy of logk), 
#    section 4 of paper or look at SNE paper <http://www.cs.toronto.edu/~hinton/absps/sne.pdf>
# - compute vol(a,b) (equation 9) 
# - compute dsigma(a,b) (proposition 13)
# - integrate vol(a,b)dsigma(a,b) over an interval of a, b with monte carlo numerical integration, a and b are sampled so that
#    1/a and 1/b are in the interval [ks, k]

#Talk : http://techtalks.tv/talks/an-information-geometry-of-statistical-manifold-learning/61044/

In [378]:
def construct_p(S):
    """
    S is a symmetric distance matrix
    """
    norm = np.exp(-S).sum(axis=1) - 1 # minus 1 to avoid counting exp(-S_ii) for each row i
    p = np.exp(-S) 
    p = p - np.eye(p.shape[0]) # minis 1 at diagonals to assign zero proba to all p_ii
    p = p / norm[:, None]
    return p

def construct_p_univariate(s, idx):
    norm = np.exp(-s).sum() - 1
    p = np.exp(-s)
    p = p / norm
    p[idx] = 0
    return p

def entropy_p_univariate(s, idx):
    log_norm = logsumexp(-s)
    log_p = - s - log_norm
    log_p[idx] = 0
    ent = (np.exp(log_p) * s).sum() + log_norm
    return ent

In [426]:
def construct_Gcov(S_Y, S_Z, S, P_S, neighb_S):
    """
    Used to construct 'g_ab' in the paper
    """
    dist, ind = neighb_S.kneighbors(S)
    dist_Y = np.take(S_Y, ind)
    dist_Z = np.take(S_Z, ind)
    P_S = np.take(P_S, ind)
    Gcov = ((P_S * (dist_Y * dist_Z)).sum(axis=1) - 
            (P_S * dist_Y).sum(axis=1) * (P_S * dist_Z).sum(axis=1))
    return Gcov

def construct_Gvar(S_Y, S, P_S, neighb_S):
    """
    Used to construct 'g_aa' and 'g_bb' in the paper
    """
    return construct_Gcov(S_Y, S_Y, S, P_S, neighb_S)

In [429]:
def construct_tau(S, k, tau0=1, verbose=1):
    """
    find tau_i for each i which we scale the distances in row i so that the distribution in row i as entropy logk
    it is done in the paper with 'binary search' or more known as 'bisection method'. the goal of bisection method
    is to find the root of a function. what we want here for a row i is find tau_i so that :
    -\sum_j p_i(j)log p_i(j) = logk
    where p(j) is proba of j given i defined by equation 1 in paper but where the distances
    are scaled by tau_i
    
    I used a fixed point optimization algo instead of bisection.

    """
    tau = []
    for i in range(S.shape[0]):
        s = S[i]
        def f(tau_sqrt):
            return entropy_p_univariate(s * tau_sqrt**2, idx=i) - np.log(k) #+ tau_sqrt
        tau_sqrt_i = newton(f, tau0)
        tau_i = tau_sqrt_i ** 2
        if verbose > 0:
            print("row {}, entropy:{}, logk={}, tau:{}".format(i, entropy_p_univariate(s * tau_i, idx=i), np.log(k), tau_i))
        tau.append(tau_i)
    tau = np.array(tau)
    return tau

In [398]:
def vol(tau_Y, tau_Z, gvar_Y, gvar_Z, gcov_YZ):
    """
    equation 9 of the paper
    """
    cov_term = (tau_Y * tau_Z * gcov_YZ).sum()
    var_Y_term = (tau_Y**2 * gvar_Y).sum()
    var_Z_term = (tau_Z**2 * gvar_Z).sum()
    return np.sqrt(1 - (cov_term**2) / (var_Y_term * var_Z_term))

def sig(tau_Y, tau_Z, gvar_Y, gvar_Z):
    """
    after proposition 13 in the paper
    """
    var_Y_term = (tau_Y**2 * gvar_Y).sum()
    var_Z_term = (tau_Z**2 * gvar_Z).sum()
    return np.sqrt(var_Y_term) * np.sqrt(var_Z_term)

In [413]:
def build_density_func(S_Y, S_Z, tau_Y, tau_Z, NN=5):
    def compute(a, b):
        S = a * S_Y + b * S_Z
        P_S = construct_p(S)
        neighb_S = NearestNeighbors(n_neighbors=NN).fit(S)
        g_ab = construct_Gcov(S_Y, S_Z, S, P_S, neighb_S)
        g_aa = construct_Gvar(S_Y, S, P_S, neighb_S)
        g_bb = construct_Gvar(S_Z, S, P_S, neighb_S)
        vol_ab = vol(tau_Y, tau_Z, g_aa, g_bb, g_ab)
        sig_ab = sig(tau_Y, tau_Z, g_aa, g_bb)
        return vol_ab * sig_ab
    return compute

In [475]:
def integrate_2d(func, a_range, b_range, nb_iter=100, verbose=1):
    val_mean = 0
    val_sqr_mean = 0
    for i in range(nb_iter):
        a = np.random.uniform(low=a_range[0], high=a_range[1])
        b = np.random.uniform(low=b_range[0], high=b_range[1])
        val = func(a, b)
        val_mean = (val + i * val_mean) / (i + 1)
        val_sqr_mean = (val**2 + i * val_sqr_mean) / (i + 1)
    val_var = val_sqr_mean - val_mean**2
    cst = (a_range[1] - a_range[0]) * (b_range[1] - b_range[0])
    if verbose > 0:
        print('Estimator variance : {}'.format(val_var * cst**2))
    return val_mean * cst

## Example

In [500]:
NN = 100
k = 10
ks = 5

In [501]:
Y, _ = make_swiss_roll(1000)
S_Y = pairwise_distances(Y)

Z = PCA(n_components=2).fit_transform(Y)
S_Z = pairwise_distances(Z)

In [502]:
neighb_Y = NearestNeighbors(n_neighbors=NN).fit(Y)
neighb_Z = NearestNeighbors(n_neighbors=NN).fit(Z)

In [503]:
tau_Y = construct_tau(S_Y, k=k, tau0=1, verbose=1)
tau_Z = construct_tau(S_Z, k=k, tau0=1, verbose=1)

row 0, entropy:2.30258509299, logk=2.30258509299, tau:1.09032130334
row 1, entropy:2.30258509299, logk=2.30258509299, tau:1.43254846237
row 2, entropy:2.30258509299, logk=2.30258509299, tau:1.25372426934
row 3, entropy:2.30258509299, logk=2.30258509299, tau:1.93288564758
row 4, entropy:2.30258509299, logk=2.30258509299, tau:1.45943623632
row 5, entropy:2.30258509299, logk=2.30258509299, tau:1.32722171833
row 6, entropy:2.30258509299, logk=2.30258509299, tau:1.79033148002
row 7, entropy:2.30258509299, logk=2.30258509299, tau:1.75729953406
row 8, entropy:2.30258509299, logk=2.30258509299, tau:1.75504031761
row 9, entropy:2.30258509299, logk=2.30258509299, tau:1.4515197571
row 10, entropy:2.30258509299, logk=2.30258509299, tau:1.01417789569
row 11, entropy:2.30258509299, logk=2.30258509299, tau:0.983979524959
row 12, entropy:2.30258509299, logk=2.30258509299, tau:0.968792363613
row 13, entropy:2.30258509299, logk=2.30258509299, tau:1.90708710435
row 14, entropy:2.30258509299, logk=2.30258

In [504]:
get_density = build_density_func(S_Y, S_Z, tau_Y, tau_Z, NN=NN)

In [505]:
integrate_2d(get_density, (1./k, 1./ks), (1./k, 1./ks), nb_iter=10000, verbose=1)

KeyboardInterrupt: 

In [ ]:
nb = 100
a = np.linspace(1./k, 1./ks, nb)
b = np.linspace(1./k, 1./ks, nb)
density = np.empty((nb, nb))
for i, j in product(np.arange(len(a)), np.arange(len(b))):
    density[i, j] = get_density(a[i], b[j])

In [ ]:
fig = plt.figure(figsize=(20, 20))
ms = plt.matshow(density, cmap='plasma')
cb = plt.colorbar(ms)
plt.show()